# Modelagem Multidimensional

A modelagem multidimensional é uma abordagem otimizada para armazenamento e análise de dados, especialmente em cenários de Business Intelligence e Data Warehousing. Seu principal objetivo é facilitar consultas rápidas e análises eficientes, organizando os dados em **dimensões** (entidades descritivas) e **tabelas fato** (eventos quantitativos).  

Na solução proposta, foram criadas as dimensões **Cliente, Filial, Parceiro e Produto**, pois essas entidades fornecem contexto para as análises de vendas. A tabela **Fato_Vendas** centraliza os dados transacionais, armazenando métricas como valores de venda, quantidade de produtos e datas das transações. Essa estrutura permite análises detalhadas de desempenho por diferentes perspectivas, como comportamento dos clientes, impacto dos parceiros e evolução das vendas ao longo do tempo.

## Dimensão Cliente

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `datawarehouse.dim_cliente` AS
SELECT id_cliente, cliente, premium
FROM `staging.cliente` c

Query is running:   0%|          |

""


## Dimensão Filial

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `datawarehouse.dim_filial` AS
SELECT
  f.id_filial,
  f.filial,
  m.municipio,
  u.uf
FROM `staging.filial` f
INNER JOIN `staging.municipio` m ON (m.id_municipio = f.id_municipio)
INNER JOIN `staging.uf` u ON (u.id_uf = m.id_uf)


## Dimensão Parceiro

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `datawarehouse.dim_associado` AS
SELECT id_associado, associado
FROM `staging.associado`

## Dimensão Item

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `datawarehouse.dim_item` AS
SELECT
  id_item,
  item,
  subcategoria,
  categoria,
  percentual
FROM `staging.item` p
INNER JOIN `staging.subcategoria` sc ON (sc.id_subcategoria = p.id_subcategoria)
INNER JOIN `staging.categoria` c ON (c.id_categoria = sc.id_categoria)

## Fato Venda

In [ ]:
%%bigquery
CREATE TABLE `datawarehouse.fato_venda`
PARTITION BY DATE(data_venda) AS
SELECT
  v.id_venda,
  iv.id_item,
  v.id_cliente,
  v.id_filial,
  v.id_associado,
  --venda,
  v.data_venda,
  iv.qtde,
  iv.vr_unitario,
  v.vr_total_pago
FROM `staging.venda` v
JOIN `staging.item_venda` iv ON iv.id_venda = v.id_venda

# Conferindo a quantidade de dados inseridos

In [ ]:
#from google.cloud import bigquery
#client = bigquery.Client()

In [ ]:
tabelas = ["cliente",
            "filial",
            "parceiro",
            "produto"
          ]
#"fato_pedido"

## Dimensões

In [ ]:
for table in tabelas:
  query = f"""
              SELECT COUNT(*) as qtdade
              FROM `datawarehouse.dim_{table}`;
          """
  #f"desafio-luizalabs-452215.dataset.{table}"

  job_config = bigquery.QueryJobConfig(allow_large_results=True)  # Permite resultados grandes
  query_job = client.query(query, job_config=job_config)

  df = query_job.to_dataframe()  # Converte para Pandas

  print(f"Tabela: dim_{table} - recebeu {df['qtdade'][0]} linhas inseridas.")

## Fato Venda

In [ ]:
query = f"""
            SELECT COUNT(*) as qtdade
            FROM `datawarehouse.fato_vendas`;
        """
job_config = bigquery.QueryJobConfig(allow_large_results=True)  # Permite resultados grandes
query_job = client.query(query, job_config=job_config)

df = query_job.to_dataframe()  # Converte para um dataframe -

print(f"Tabela: fato_pedido - {df['qtdade'][0]} linhas inseridas.")